<a href="https://colab.research.google.com/github/VIPlearner/csv-files/blob/main/XGBboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xgboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, Normalizer, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV, train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,BaggingRegressor,VotingRegressor,StackingRegressor
from catboost import CatBoostRegressor

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/VIPlearner/csv-files/main/Train.csv", index_col="VehicleID")
test_df=pd.read_csv("https://raw.githubusercontent.com/VIPlearner/csv-files/main/Test.csv")

df = df[~(np.isnan(df["Amount (Million Naira)"]))]

In [4]:
df['train']  = 1
test_df['train']  = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
all_df = pd.concat([df.drop('Amount (Million Naira)',axis  =  1),test_df]).reset_index(drop = True)
all_df['Year'] = all_df['Year'].str.replace(',', '').astype(float)

In [6]:
#dropping irrelevant features
#all_df.drop(['Colour'], axis = 1, inplace = True)
#all_df.drop(['Maker'], axis = 1, inplace = True)
# all_df.drop(['Type'], axis = 1, inplace = True)
# all_df.drop(['Model'], axis = 1, inplace = True)
all_df.drop(['Location'], axis = 1, inplace = True)
all_df.drop(['Distance'], axis = 1, inplace = True)
all_df.drop(['VehicleID'], axis = 1, inplace = True)
all_df.head()

,Maker,Model,Year,Colour,Type,train
0,Honda,Accord Coupe EX V-6,2011.0,Silver,Nigerian Used,1
1,Hyundai,Sonata,2012.0,Silver,Nigerian Used,1
2,Lexus,RX 350,2010.0,Red,Foreign Used,1
3,Mercedes-Benz,GLE-Class,2017.0,Blue,Foreign Used,1
4,Toyota,Highlander,2002.0,Red,Nigerian Used,1


In [7]:
#Creating a count vectorizer for all the strings 
mashup = [''.join(all_df[['Year', 'Maker', 'Model', 'Colour', 'Type']].astype(str).iloc[i].values) for i in range(len(all_df))]
### 
mashup_cleaned = [mashup[i].strip().replace(',','') for i in range(len(mashup))]

Vectorizer= CountVectorizer(
                analyzer = "char",  
                tokenizer = None,  
                preprocessor = None, 
                stop_words = None,  
                max_features = 5000,
)
Vectorizer.fit(mashup_cleaned)
letter_array = Vectorizer.transform(mashup_cleaned).toarray()
vocab = Vectorizer.vocabulary_
letter_data = pd.DataFrame(letter_array,columns=vocab)
all_df = pd.concat([all_df,letter_data],axis = 1)
all_df.head()

,Maker,Model,Year,Colour,Type,train,2,0,1,.,...,w,4,8,+,(,),q,j,/,&
0,Honda,Accord Coupe EX V-6,2011.0,Silver,Nigerian Used,1,4,0,0,0,...,0,3,2,0,2,2,0,1,0,0
1,Hyundai,Sonata,2012.0,Silver,Nigerian Used,1,1,0,0,0,...,0,2,3,1,2,1,0,0,1,0
2,Lexus,RX 350,2010.0,Red,Foreign Used,1,2,0,0,0,...,0,3,2,0,2,0,0,2,0,0
3,Mercedes-Benz,GLE-Class,2017.0,Blue,Foreign Used,1,1,0,0,0,...,0,2,4,0,2,0,0,0,0,1
4,Toyota,Highlander,2002.0,Red,Nigerian Used,1,1,0,0,0,...,0,3,1,2,1,0,0,0,1,0


In [8]:
all_df['Type'] = all_df.Type.astype('category')
all_df['Maker'] = all_df.Maker.astype('category')
all_df['Model'] = all_df.Model.astype('category')
all_df['Colour'] = all_df.Colour.astype('category')
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9249 entries, 0 to 9248
Data columns (total 50 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   Maker   9249 non-null   category
 1   Model   9249 non-null   category
 2   Year    9226 non-null   float64 
 3   Colour  9249 non-null   category
 4   Type    8999 non-null   category
 5   train   9249 non-null   int64   
 6   2       9249 non-null   int64   
 7   0       9249 non-null   int64   
 8   1       9249 non-null   int64   
 9   .       9249 non-null   int64   
 10  h       9249 non-null   int64   
 11  o       9249 non-null   int64   
 12  n       9249 non-null   int64   
 13  d       9249 non-null   int64   
 14  a       9249 non-null   int64   
 15  c       9249 non-null   int64   
 16  r       9249 non-null   int64   
 17          9249 non-null   int64   
 18  u       9249 non-null   int64   
 19  p       9249 non-null   int64   
 20  e       9249 non-null   int64   
 21  x       9249 n

In [9]:
train_df = all_df[all_df.train == 1].drop('train',axis = 1)
t_df = all_df[all_df.train == 0].drop('train',axis = 1)
X = train_df
y = df["Amount (Million Naira)"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [10]:
r1 = XGBRegressor(random_state = 42, max_depth = 3, learning_rate = 0.1, colsample_bytree=0.1,  n_estimators=3000)
r2 = RandomForestRegressor(random_state=42, max_features = 'auto', n_estimators = 3000)
r3 = CatBoostRegressor(verbose=False,random_state = 42)
model = VotingRegressor([('r1', r1),('rf', r2),('r3',r3)],weights=(1,1,1))


In [11]:
#numeric_features = ["Year", "Type"]
year_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")), ("scaler", MinMaxScaler())]
)

distance_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())]
)

type_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")), ("encoder", OneHotEncoder(handle_unknown="ignore"))]
)
categorical_features = ["Model", 'Maker', 'Colour', ]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("year", year_transformer, ["Year"]),
        # ("distance", year_transformer, ["Distance"]),
        ("cat", categorical_transformer, categorical_features),
        ("type_cat", type_transformer, ["Type"]),  
    ]
)



In [12]:
pipeline = Pipeline(
  steps=[("preprocessor", preprocessor), ("estimator", CatBoostRegressor(verbose=False, random_state = 42))]
)



In [13]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'estimator', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__verbose_feature_names_out', 'preprocessor__year', 'preprocessor__cat', 'preprocessor__type_cat', 'preprocessor__year__memory', 'preprocessor__year__steps', 'preprocessor__year__verbose', 'preprocessor__year__imputer', 'preprocessor__year__scaler', 'preprocessor__year__imputer__add_indicator', 'preprocessor__year__imputer__copy', 'preprocessor__year__imputer__fill_value', 'preprocessor__year__imputer__missing_values', 'preprocessor__year__imputer__strategy', 'preprocessor__year__imputer__verbose', 'preprocessor__year__scaler__clip', 'preprocessor__year__scaler__copy', 'preprocessor__year__scaler__feature_range', 'preprocessor__cat__categories', 'preprocessor__cat__drop', 'preprocessor__cat__dtype', 'preprocessor__cat__handle_unkn

In [14]:
parameters = {#'estimator__alpha': [1],
              #'estimator':[DecisionTreeRegressor()],
              'estimator__iterations': np.arange(10, 500, 10),
              'estimator__depth': np.arange(2, 16, 2),
              'estimator__learning_rate': [0.01, 0.05, 0.1]
}

In [ ]:
grid = GridSearchCV(pipeline, parameters, cv=2)

grid.fit(X_train, y_train)            
  

In [ ]:
grid.best_params_

In [ ]:
print("model score: %.3f" % grid.score(X_test, y_test))

In [ ]:
y_pred = grid.predict(X_test)
errors = mean_squared_error(y_test, y_pred, squared = False) 
print(errors)

In [ ]:
test_df=pd.read_csv("https://raw.githubusercontent.com/VIPlearner/csv-files/main/Test.csv")
test_df.head()
# dropping irrelevant features
#test_df.drop(['Colour'], axis = 1, inplace = True)
#test_df.drop(['Maker'], axis = 1, inplace = True)
#test_df.drop(['Model'], axis = 1, inplace = True)
# test_df.drop(['Location'], axis = 1, inplace = True)
test_df.drop(['Distance'], axis = 1, inplace = True)
test_df['Year'] = test_df['Year'].str.replace(',', '').astype(float)
# test_df['Distance'] = test_df['Distance'].str.replace(',', '').astype(float)
test_df['Location'] = test_df.Location.astype('category')
test_df['Type'] = test_df.Type.astype('category')
test_df['Maker'] = test_df.Maker.astype('category')
test_df['Model'] = test_df.Model.astype('category')
test_df['Colour'] = test_df.Colour.astype('category')


In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061 entries, 0 to 2060
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   VehicleID  2061 non-null   object  
 1   Location   2061 non-null   category
 2   Maker      2061 non-null   category
 3   Model      2061 non-null   category
 4   Year       2059 non-null   float64 
 5   Colour     2061 non-null   category
 6   Type       2007 non-null   category
dtypes: category(5), float64(1), object(1)
memory usage: 67.5+ KB


In [52]:
test_y_pred = grid.predict(t_df)

In [76]:
test_pred_df = pd.DataFrame({'VehicleID' : test_df['VehicleID'], 'Amount (Million Naira)': test_y_pred})
test_pred_df.head

<bound method NDFrame.head of      VehicleID  Amount (Million Naira)
0     VHL18518                5.886045
1     VHL17149                6.267722
2     VHL10927                4.021592
3     VHL12909                4.723862
4     VHL12348                8.772875
...        ...                     ...
2056  VHL17903               22.895220
2057  VHL14018                6.867508
2058  VHL17473                7.693291
2059  VHL11480                9.371327
2060  VHL13881                4.127173

[2061 rows x 2 columns]>

In [78]:
test_pred_df.to_csv('47.csv', encoding='utf-8', index=False)

In [79]:
from google.colab import files
files.download("47.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
help(XGBRegressor())

Help on XGBRegressor in module xgboost.sklearn object:

class XGBRegressor(XGBModel, sklearn.base.RegressorMixin)
 |  XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=100, verbosity=1, silent=None, objective='reg:linear', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, colsample_bynode=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, importance_type='gain', **kwargs)
 |  
 |  Implementation of the scikit-learn API for XGBoost regression.
 |  
 |  Parameters
 |  ----------
 |  max_depth : int
 |      Maximum tree depth for base learners.
 |  learning_rate : float
 |      Boosting learning rate (xgb's "eta")
 |  n_estimators : int
 |      Number of trees to fit.
 |  verbosity : int
 |      The degree of verbosity. Valid values are 0 (silent) - 3 (debug).
 |  silent : boolean
 |      Whether to print messages while running boosti